In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt 
from collections import Counter 
from typing import Literal, Union, Optional, List
import utils

In [3]:
libra_df = pd.read_csv('data/Libra.csv')
libra_G = nx.from_pandas_edgelist(libra_df,source = 'Source', target = 'Target', edge_attr=True,create_using=nx.DiGraph)
print('The libra |N|:',libra_G.number_of_nodes())


rabo_G = nx.read_edgelist('data/rabo_easylabel.edgelist',data=(("Total",float),("Count", int)), create_using=nx.DiGraph)
print('The rabo |N|:',rabo_G.number_of_nodes())
rabo_df = nx.to_pandas_edgelist(rabo_G,source='Source',target='Target')

The libra |N|: 385100
The rabo |N|: 1624030


In [13]:
def show_basics(G,df):
    N  = G.number_of_nodes()
    print('|N|: ', G.number_of_nodes())
    print('|E|: ', G.number_of_edges())
    print('|W_T|: ', df['Total'].sum())
    print('|W_C|: ', df['Count'].sum())
    print('reciprocity: ', nx.reciprocity(G))
    print('average clustering coef: ',nx.approximation.average_clustering(nx.Graph(G),trials=int(0.01*N),seed=101))


    degrees = [degree for node, degree in G.degree()]
    avg_degree = sum(degrees) / G.number_of_nodes()
    print('average degree: ', avg_degree)

    df.groupby('Source').size().reset_index()
    source = set(df.groupby('Source').size().reset_index()['Source'])
    target = set(df.groupby('Target').size().reset_index()['Target'])
    print('number of nodes who have at least one out_degree:',len(source))
    print('number of nodes who have at least one in_degree:',len(target))
    print('intersection of the above two: ', len(source&target))





In [15]:
show_basics(rabo_G,rabo_df)

|N|:  1624030
|E|:  3823167
|W_T|:  291294725973.0
|W_C|:  110609151
reciprocity:  0.26319697779354134
average clustering coef:  0.020997536945812807
average degree:  4.708246768840477
number of nodes who have at least one out_degree: 1300207
number of nodes who have at least one in_degree: 690490
intersection of the above two:  366667
